In [1]:
# 기본 패키지 불러오기 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use("seaborn")
sns.set(font_scale=1)
sns.set_style("whitegrid")

# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 맥 OS 폰트 깨짐 방지 코드
from matplotlib import rc
import matplotlib as mpl 
import matplotlib.font_manager as fm 

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## Inside Airbnb: London Data

### 컬럼 속성 이해

In [2]:
london_lists_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings.csv')
london_lists_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings_outline.csv')
london_neighbourhoods_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/neighbourhoods.csv')
london_reviews_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews.csv')
london_reviews_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews_outline.csv')

In [3]:
london_lists = london_lists_raw.copy()
london_lists

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,312761,https://www.airbnb.com/rooms/312761,20240319050633,2024-03-21,city scrape,Spacious Central London Apartment by Hoxton Sq...,"Very central location, in the middle of Shored...",Everything is so convenient and the area is al...,https://a0.muscache.com/pictures/miso/Hosting-...,1608226,...,5.00,4.89,4.93,NaN,f,1,1,0,0,0.86
1,13913,https://www.airbnb.com/rooms/13913,20240319050633,2024-03-20,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.83,4.70,4.70,NaN,f,3,2,1,0,0.25
2,15400,https://www.airbnb.com/rooms/15400,20240319050633,2024-03-20,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,...,4.83,4.93,4.74,NaN,f,1,1,0,0,0.54
3,159736,https://www.airbnb.com/rooms/159736,20240319050633,2024-03-20,city scrape,A double Room 5mins from King's College Hospital,Calm sunny double room with a queen size bed a...,We love that in Loughborough Junction we live ...,https://a0.muscache.com/pictures/1067303/d2300...,766056,...,4.89,4.34,4.66,NaN,f,4,0,4,0,0.62
4,165336,https://www.airbnb.com/rooms/165336,20240319050633,2024-03-21,city scrape,Charming Flat in Notting Hill,A stylish apartment close to Portobello market...,"Notting Hill has many cafes, bars and restaura...",https://a0.muscache.com/pictures/60757460/47f8...,761400,...,4.96,4.87,4.71,NaN,f,1,1,0,0,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90847,1115272743905612628,https://www.airbnb.com/rooms/1115272743905612628,20240319050633,2024-03-19,city scrape,Primrose Hill Studio #25,A place to spread out in with extra wiggle roo...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,565833376,...,NaN,NaN,NaN,NaN,t,48,48,0,0,NaN
90848,1115272767893608505,https://www.airbnb.com/rooms/1115272767893608505,20240319050633,2024-03-19,city scrape,Primrose Hill Studio #27,"Two-storey living, unwind downstairs and rest ...",NaN,https://a0.muscache.com/pictures/prohost-api/H...,565833376,...,NaN,NaN,NaN,NaN,t,48,48,0,0,NaN
90849,1115285644506865913,https://www.airbnb.com/rooms/1115285644506865913,20240319050633,2024-03-21,city scrape,Quad Bedroom with External Bathroom,The Chapter Hotels group had been in the hotel...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,444801189,...,NaN,NaN,NaN,NaN,t,33,10,23,0,NaN
90850,1115338006572763275,https://www.airbnb.com/rooms/1115338006572763275,20240319050633,2024-03-20,city scrape,Grosvenor Collection,This bright and spacious three-bedroom penthou...,"The area itself is a vibrant neighbourhood, of...",https://a0.muscache.com/pictures/prohost-api/H...,514234035,...,NaN,NaN,NaN,NaN,t,52,51,1,0,NaN


In [4]:
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            90852 non-null  int64  
 1   listing_url                                   90852 non-null  object 
 2   scrape_id                                     90852 non-null  int64  
 3   last_scraped                                  90852 non-null  object 
 4   source                                        90852 non-null  object 
 5   name                                          90852 non-null  object 
 6   description                                   87851 non-null  object 
 7   neighborhood_overview                         47521 non-null  object 
 8   picture_url                                   90842 non-null  object 
 9   host_id                                       90852 non-null 

In [5]:
london_lists.columns.to_frame().reset_index(drop=True).rename(columns={0:'column name'}).to_csv('./NameofColumns_rawdata.csv')

In [6]:
london_lists.host_verifications.unique()

array(["['email', 'phone']", "['phone']",
       "['email', 'phone', 'work_email']", "['email']", '[]',
       "['phone', 'work_email']", "['email', 'phone', 'photographer']",
       'None', "['email', 'work_email']"], dtype=object)

### host_identity_verified 컬럼

In [7]:
# 호스트 신원 검증 여부 컬럼의 유니크 값 확인 
london_lists.host_identity_verified.unique()

array(['t', 'f', nan], dtype=object)

In [8]:
# 전체 행 개수 확인 : 90,852개
print(london_lists.shape)
print()

# 호스트의 신원이 검증된 숙소의 개수 : 79,553개
host_identity_verified_t = london_lists.host_identity_verified == 't'
print(london_lists[host_identity_verified_t].shape)
print()

# 호스트의 신원이 검증되지 않은 숙소의 개수 : 11,294개
host_identity_verified_f = london_lists.host_identity_verified == 'f'
print(london_lists[host_identity_verified_f].shape)
print()

# 신원 검증 여부가 null값인 개수 : 5개 
print(london_lists.host_identity_verified.isnull().sum())
print()

# 누락 여부 확인 : 누락 없음
79553 + 11294 + 5

(90852, 75)

(79553, 75)

(11294, 75)

5



90852

neighbourhood /neighbourhood_cleansed /neighbourhood_group_cleansed 컬럼


In [9]:
# neighbourhood: 숙소가 위치한 넓은 지역명 컬럼 유니크값 확인 

london_lists.neighbourhood.unique()

array(['London, England, United Kingdom',
       'Islington, Greater London, United Kingdom',
       'London, United Kingdom', 'Lambeth, England, United Kingdom',
       'Kensington, England, United Kingdom', nan,
       'Greater London, England, United Kingdom',
       'London, Fitzrovia, United Kingdom',
       " St Margaret's, Isleworth, Middlesex, United Kingdom",
       'Richmond, United Kingdom', 'Paddington, England, United Kingdom',
       'Oval, London, United Kingdom', 'East Finchley, United Kingdom',
       'Enfield, United Kingdom', 'Fulham, London, United Kingdom',
       'Chingford, London, United Kingdom',
       'Richmond, England, United Kingdom',
       'Hounslow, Greater London, United Kingdom',
       'Hackney, London, United Kingdom',
       'Barking, London, United Kingdom',
       'Walthamstow, Greater London, United Kingdom',
       'Kensington, Greater London, United Kingdom',
       'Islington, London, United Kingdom',
       'London, Greater London, United Ki

In [10]:
# neighbourhood 컬럼의 유니크한 값 개수 확인 및 null 값 개수 확인 
print(london_lists.neighbourhood.nunique())
print(london_lists.neighbourhood.isnull().sum())

865
43330


In [11]:
# neighbourhood_cleansed의 유니크값 확인

london_lists.neighbourhood_cleansed.unique()

array(['Hackney', 'Islington', 'Kensington and Chelsea', 'Lambeth',
       'Brent', 'Southwark', 'Westminster', 'Wandsworth', 'Tower Hamlets',
       'Richmond upon Thames', 'Newham', 'Camden', 'Haringey', 'Enfield',
       'Hammersmith and Fulham', 'Waltham Forest', 'Barnet', 'Hounslow',
       'Barking and Dagenham', 'Croydon', 'Lewisham', 'Ealing', 'Merton',
       'City of London', 'Havering', 'Greenwich', 'Hillingdon', 'Bromley',
       'Redbridge', 'Harrow', 'Bexley', 'Kingston upon Thames', 'Sutton'],
      dtype=object)

In [12]:
# neighbourhood_cleansed의 유니크값 개수 확인 및 null값 개수 확인

print(london_lists.neighbourhood_cleansed.nunique())
print(london_lists.neighbourhood_cleansed.isnull().sum())

33
0


In [13]:
# neighbourhood_group_cleansed의 null 값 개수 확인 

london_lists.neighbourhood_group_cleansed.isnull().sum()

90852

* neighbourhood 컬럼 유니크값이 865개로 지역별 데이터를 바탕으로한 인사이트를 얻기 어려움 
* neighbourhood_group_cleansed 행 전체가 null값 >> 로우 데이터에서도 삭제하는 것이 좋을 것.

<br />

* neighbourhood_cleansed 컬럼이 유니크값이 33개로 그나마 지역별 인사이트를 얻는 데 유용할 것.

### property_type 관련 컬럼

In [14]:
london_lists.property_type.unique()

array(['Entire rental unit', 'Private room in rental unit',
       'Entire townhouse', 'Entire home', 'Private room in home',
       'Entire guesthouse', 'Private room in townhouse', 'Entire condo',
       'Private room in condo', 'Room in serviced apartment',
       'Room in aparthotel', 'Entire serviced apartment',
       'Private room in loft', 'Private room',
       'Private room in bed and breakfast', 'Entire guest suite',
       'Houseboat', 'Private room in serviced apartment',
       'Private room in bungalow', 'Private room in guesthouse',
       'Shared room in home', 'Entire loft', 'Entire cabin',
       'Private room in yurt', 'Boat', 'Shared room in condo',
       'Private room in guest suite', 'Entire cottage',
       'Private room in villa', 'Private room in cottage', 'Tiny home',
       'Entire place', 'Private room in houseboat', 'Shared room in loft',
       'Room in bed and breakfast', 'Shared room in rental unit',
       'Private room in casa particular', 'Shepherd’

In [15]:
print(london_lists.property_type.nunique())
print(london_lists.property_type.isnull().sum())

99
0


In [16]:
print(london_lists.room_type.unique())
print()
print(london_lists.room_type.isnull().sum())

['Entire home/apt' 'Private room' 'Hotel room' 'Shared room']

0


* property_type 보다는 room_type이 그룹에 따른 인사이트를 얻기에 적합함 

In [46]:
london_lists.amenities.unique()

array(['["Toaster", "Room-darkening shades", "Coffee maker: french press", "Wifi", "AC - split type ductless system", "Dishwasher", "Cleaning products", "Luggage dropoff allowed", "Dishes and silverware", "Hot water kettle", "Free dryer \\u2013 In unit", "Extra pillows and blankets", "Free washer \\u2013 In unit", "Hot water", "Kitchen", "Laundromat nearby", "Bathtub", "Clothing storage", "Wine glasses", "Central heating", "Bed linens", "Single level home", "Portable fans", "Microwave", "Private patio or balcony", "Essentials", "Freezer", "Iron", "Bosch stainless steel gas stove", "Refrigerator", "Hangers", "Cleaning available during stay", "Carbon monoxide alarm", "Coffee", "55\\" HDTV with Netflix, premium cable, standard cable", "Smoke alarm", "Long term stays allowed", "Dedicated workspace", "Cooking basics", "Paid parking off premises", "Hair dryer", "Dining table", "Ethernet connection", "Drying rack for clothing", "Oven", "Books and reading material"]',
       '["Toaster", "Room

In [47]:
london_lists.amenities.nunique()

81129

* amenities는 유니크 값이 워낙 다양하게 있기 때문에 이 형태로는 사용하기 어려움 

In [28]:
# 가격이 누락된 숙소 수 : 32,967개

london_lists.price.isnull().sum()

32967

### 리뷰 관련 컬럼

In [36]:
# 리뷰 전체 갯수의 null 값 = 0개
london_lists.number_of_reviews.isnull().sum()

0

In [37]:
# 리뷰 전체 개수가 0인 컬럼의 개수 = 24,983개

london_lists[london_lists.number_of_reviews == 0]

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
9,170524,https://www.airbnb.com/rooms/170524,20240319050633,2024-03-20,city scrape,STUNNING APT IN ST JAMES,NaN,NaN,https://a0.muscache.com/pictures/1077542/ba973...,194769,...,NaN,NaN,NaN,NaN,f,4,3,1,0,NaN
35,318986,https://www.airbnb.com/rooms/318986,20240319050633,2024-03-22,previous scrape,Central London flat for Olympics,NaN,NaN,https://a0.muscache.com/pictures/4746749/6bfa7...,1637212,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
45,42692,https://www.airbnb.com/rooms/42692,20240319050633,2024-03-22,previous scrape,Fabulous flat w garden and bkfst!,Holland Park/Notting Hill. Guests have access ...,The neighborhood of Holland Park borders Notti...,https://a0.muscache.com/pictures/239496/630e38...,186510,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
49,44384,https://www.airbnb.com/rooms/44384,20240319050633,2024-03-19,city scrape,DOUBLE ROOM IN A PENTHOUSE APARTMEN,A stunning double room with great views.<br />...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,194769,...,NaN,NaN,NaN,NaN,t,4,3,1,0,NaN
56,324055,https://www.airbnb.com/rooms/324055,20240319050633,2024-03-22,previous scrape,worth to stay.you will never regret,NaN,NaN,https://a0.muscache.com/pictures/3526645/f9c03...,1664001,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90847,1115272743905612628,https://www.airbnb.com/rooms/1115272743905612628,20240319050633,2024-03-19,city scrape,Primrose Hill Studio #25,A place to spread out in with extra wiggle roo...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,565833376,...,NaN,NaN,NaN,NaN,t,48,48,0,0,NaN
90848,1115272767893608505,https://www.airbnb.com/rooms/1115272767893608505,20240319050633,2024-03-19,city scrape,Primrose Hill Studio #27,"Two-storey living, unwind downstairs and rest ...",NaN,https://a0.muscache.com/pictures/prohost-api/H...,565833376,...,NaN,NaN,NaN,NaN,t,48,48,0,0,NaN
90849,1115285644506865913,https://www.airbnb.com/rooms/1115285644506865913,20240319050633,2024-03-21,city scrape,Quad Bedroom with External Bathroom,The Chapter Hotels group had been in the hotel...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,444801189,...,NaN,NaN,NaN,NaN,t,33,10,23,0,NaN
90850,1115338006572763275,https://www.airbnb.com/rooms/1115338006572763275,20240319050633,2024-03-20,city scrape,Grosvenor Collection,This bright and spacious three-bedroom penthou...,"The area itself is a vibrant neighbourhood, of...",https://a0.muscache.com/pictures/prohost-api/H...,514234035,...,NaN,NaN,NaN,NaN,t,52,51,1,0,NaN


In [39]:
# 리뷰 스코어 레이팅 null 값 개수 확인 : 24,983개 
london_lists.review_scores_rating.isnull().sum()

24983

* 리뷰 스코어 레이팅 null 값 개수 = 리뷰가 0개인 숙소 수

In [40]:
# 리뷰 평점이 0인 숙소의 개수 : 4개
london_lists[london_lists.review_scores_rating == 0]

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
478,486501,https://www.airbnb.com/rooms/486501,20240319050633,2024-03-22,previous scrape,Bright 2 Bed Flat by Queens Park,Light and spacious 2 double bedroom flat next ...,NaN,https://a0.muscache.com/pictures/5781343/93d3e...,979982,...,5.0,4.0,3.0,NaN,t,1,1,0,0,0.01
856,698818,https://www.airbnb.com/rooms/698818,20240319050633,2024-03-22,previous scrape,Comfy Studio Private Garden/Kitchen,NaN,NaN,https://a0.muscache.com/pictures/9618563/79ad5...,3582753,...,5.0,5.0,4.0,NaN,f,1,1,0,0,0.01
982,760374,https://www.airbnb.com/rooms/760374,20240319050633,2024-03-22,previous scrape,London room for rent - good value!,I have a spare room in a small house.<br /><br...,I live in a residential area. My home is part ...,https://a0.muscache.com/pictures/10571364/d359...,4007960,...,NaN,NaN,NaN,NaN,f,1,0,1,0,0.01
20192,21845656,https://www.airbnb.com/rooms/21845656,20240319050633,2024-03-21,previous scrape,Comfortable apartment in central Chiswick,"Sole use of light, airy and comfortable apartm...",Leafy residential area yet only 25 minutes int...,https://a0.muscache.com/pictures/01d8135a-3aa4...,45795526,...,5.0,5.0,5.0,NaN,f,2,1,1,0,0.01


In [42]:
# 월 평균 리뷰 개수 컬럼의 null 값 확인 : 24,983개
london_lists['reviews_per_month'].isnull().sum()

24983

### 호스트가 운영하는 숙소 총 개수

In [44]:
# null값 개수 확인
london_lists.calculated_host_listings_count.isnull().sum()

0

In [45]:
london_lists.calculated_host_listings_count.describe()

count    90852.000000
mean        18.495927
std         65.205497
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max        558.000000
Name: calculated_host_listings_count, dtype: float64

In [49]:
london_lists.has_availability.unique()

array(['t', nan, 'f'], dtype=object)

In [ ]:
london_lists.has_availability.unique()